In [2]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 83.6 MB/s eta 0:00:00


In [3]:
from pulp import *
import pandas as pd
import pickle as pkl
import numpy as np

### Insert the information about the players who are in the playing 11 (along with the substitutes) in the format as mentioned below. The expected points as of now can be entered to be average points of the player, but should be aimed for having a better predictions for the expected points.

In [4]:
# Insert the information of the players in the playing team!
# The points are the avg expected points directly picked from the dream 11 portal 

players_info = {  'J Buttler':{'points':49.85, 'Team':'RR', 'cat':'wk', 'cost':9}, 
                  'S Samson':{'points':49.62, 'Team':'RR', 'cat':'wk', 'cost':8.5}, 
                  'J Sharma':{'points':35.69, 'Team':'PBKS', 'cat':'wk', 'cost':7.5}, 
                  'P Singh':{'points':42.15, 'Team':'PBKS', 'cat':'wk', 'cost':7},

                  'S Dhawan':{'points':53.5, 'Team':'PBKS', 'cat':'bat', 'cost':10}, 
                  'Y Jaiswal':{'points':71.15, 'Team':'RR', 'cat':'bat', 'cost':9}, 
                  'S Hetmyer':{'points':38.23, 'Team':'RR', 'cat':'bat', 'cost':8.5}, 
                  'J Root':{'points':13, 'Team':'RR', 'cat':'bat', 'cost':7}, 
                  'D Padikkal':{'points':33, 'Team':'RR', 'cat':'bat', 'cost':7}, 
                  'S Khan':{'points':20.15, 'Team':'PBKS', 'cat':'bat', 'cost':6.5}, 
                  'D Jurel':{'points':21.58, 'Team':'RR', 'cat':'bat', 'cost':6.5},
                  'S Taide':{'points':46.33, 'Team':'PBKS', 'cat':'bat', 'cost':6},

                  'R Ashwin':{'points':41.23, 'Team':'RR', 'cat':'ar', 'cost':8.5},
                  'L Livingstone':{'points':58.88, 'Team':'PBKS', 'cat':'ar', 'cost':8.5}, 
                  'S Curran':{'points':46.31, 'Team':'PBKS', 'cat':'ar', 'cost':8.5},

                  'Y Chahal':{'points':50.69, 'Team':'RR', 'cat':'bowl', 'cost':9}, 
                  'A Singh':{'points':38.31, 'Team':'PBKS', 'cat':'bowl', 'cost':9}, 
                  'A Zampa':{'points':42.2, 'Team':'RR', 'cat':'bowl', 'cost':8}, 
                  'K Rabada':{'points':27, 'Team':'PBKS', 'cat':'bowl', 'cost':8}, 
                  'R Chahar':{'points':21.92, 'Team':'PBKS', 'cat':'bowl', 'cost':7.5}, 
                  'N Ellis':{'points':42.11, 'Team':'PBKS', 'cat':'bowl', 'cost':7.5}, 
                  'S Sharma':{'points':32.91, 'Team':'RR', 'cat':'bowl', 'cost':7}, 
                  'H Brar':{'points':31., 'Team':'PBKS', 'cat':'bowl', 'cost':7}}




### Optimization formulation -- 

Variables:

x_i: Binary variable representing whether player i is selected (x_i = 1) or not (x_i = 0).

y_i: Binary variable representing whether player i is selected as the captain (y_i = 1) or not (y_i = 0).

k_i: Binary variable representing whether player i is selected as the vice captain (k_i = 1) or not (k_i = 0).

p_i: Expected points for player i.

c_i: Cost associated with player i.

t_i: Team associated with player i (t_i = 'A' or t_i = 'B').

**Objective -- ** Maximize the expected points: 

*Maximize*: sum_i ((p_i \* x_i) + (p_i \* y_i) + 0.5\*(p_i \* k_i)) = sum_i (p_i \* (x_i + y_i + 0.5\*k_i))

Subject to --- 
1. Total cost constraint: sum(c_i * x_i) <= 100

2. Player selection constraint: sum(x_i) = 11

3. Team constraint: sum(x_i * (t_i = 'A')) <= 7 || sum(x_i * (t_i = 'B')) <= 7

4. Captain constraint: sum(y_i) = 1

5. Vice Captain constraint: sum(k_i) = 1

6. Captain needs to be only from the selected players: y_i <= x_i

7. Vice-captain needs to be only from the selected players: k_i <= x_i

8. A player cannot be captain and vice-captain at the same time: y_i + k_i <= 1


In [9]:
players_list = [p for p in players_info]
points_array = [players_info[p]['points'] for p in players_list]
cost_array = [players_info[p]['cost'] for p in players_list]

#--------------------- Define the optimization variables --------

z_variables = []
z_variables_captain = []
z_variables_vice_captain = []

M = len(players_list)
j = 0
for i in range(M):      
    x = LpVariable('x_{}_{}'.format(i,j), 0, 1, cat='Integer')
    z_variables.append(x)
    
    y = LpVariable('y_{}_{}'.format(i,j), 0, 1, cat='Integer')    
    z_variables_captain.append(y)
    
    k = LpVariable('k_{}_{}'.format(i,j), 0, 1, cat='Integer')    
    z_variables_vice_captain.append(k)


#--------------------- Define the optimization problem --------

problem = LpProblem('Dream team optimization', LpMaximize)

expected_points = lpDot(z_variables, points_array) + lpDot(z_variables_captain, points_array) + 0.5*lpDot(z_variables_vice_captain, points_array) 

total_cost = lpDot(z_variables, cost_array) 

problem += expected_points

#--------------------- Setting up the constraints --------

problem += total_cost <= 100.

team_RR, team_PBKS = 0, 0
for i, p in enumerate(players_list):
    if (players_info[p]['Team'] == 'RR'):
        team_RR += z_variables[i]
    elif (players_info[p]['Team'] == 'PBKS'):
        team_PBKS += z_variables[i]

problem += team_RR <= 7
problem += team_PBKS <= 7

category_constraint = {'wk':0, 'bat':0, 'bowl':0, 'ar':0}
for i, p in enumerate(players_list):
    category_constraint[players_info[p]['cat']] += z_variables[i]

for c in category_constraint:
    problem += category_constraint[c] <= 8
    problem += category_constraint[c] >= 1

problem += lpSum(z_variables) == 11
problem += lpSum(z_variables_captain) == 1
problem += lpSum(z_variables_vice_captain) == 1

for i in range(len(z_variables)):
    problem += z_variables_captain[i] <= z_variables[i]
    problem += z_variables_vice_captain[i] <= z_variables[i]
    
    problem += (z_variables_captain[i] + z_variables_vice_captain[i]) <= 1

    

print('Solving LP') 
solverdir = '/content/cbc' 

LpSolverDefault.msg = 1
solver = COIN_CMD(path=solverdir, maxSeconds = 3600) 
start = 0 
status = problem.solve(solver)

print('problem.objective.value() -- ',problem.objective.value())
status_dict = {1:"Optimal", 0:"Not Solved", -1:"Infeasible", -2:"Unbounded", -3:"Undefined"}

print ('Opt_status =', status_dict[status])

Solving LP
problem.objective.value() --  653.38
Opt_status = Optimal


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/usr/local/lib/python3.10/dist-packages/pulp/apis/coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


## Get the team output --

In [10]:
print ("Dream team based on the average performance of players in the league:")
print()

total_cost = 0
for idx,v in enumerate(z_variables):
    if (v.value() == 1): 
        total_cost += cost_array[idx]
        print (players_list[idx], end = '')
        if (z_variables_captain[idx].value() == 1): 
            print (' -- Captain', end = '')
        if (z_variables_vice_captain[idx].value() == 1): 
            print (' -- V. Captain', end = '')
        print() 

print ()
print ("Team cost -- ", total_cost)

Dream team based on the average performance of players in the league:

J Buttler
S Samson
P Singh
S Dhawan
Y Jaiswal -- Captain
S Taide
L Livingstone -- V. Captain
S Curran
Y Chahal
A Zampa
N Ellis

Team cost --  91.0
